In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

Do not ever put API's in your repositories. This one regenerates after 24 hours and doesn't have any sensitive information behind it. But do as I say, not as I do.

In [2]:
lol_watcher = LolWatcher('RGAPI-dc0706f1-4b82-4c87-8f6a-58620810a5ab')

my_region = 'na1'


In [3]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

In [4]:
friends = [lol_watcher.summoner.by_name(my_region, summoner) for summoner in summoners]

In [7]:
friends

[{'id': 'pOYthgJrELYHPeOs0onX150u6FehhlVgcmqDnNsL9lrMsY4',
  'accountId': 'W-SQGFfhtm8nKDogjf7XkKpSAcNwEAbgdWJ2K2ZXAxlTPuk',
  'puuid': '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
  'name': 'Zealfire',
  'profileIconId': 622,
  'revisionDate': 1654830407000,
  'summonerLevel': 188},
 {'id': '0O8bnnWbnEtqy4ziXhR8CVe7HXL6hXZv2aph4brxu_XT4Jc',
  'accountId': '6itspRrdvLM2APWeg_YbdAkuVd1-3OnNLDPuj1W6JRJ_h1k',
  'puuid': 'hGFiqH_c8ykOtfmYyn6Hg2WflOVI-J2Nt4HFkhdqB7ClHoCYaX4CFq146wont9KB2E3qiDHIYUv7-Q',
  'name': 'SmurfHD',
  'profileIconId': 983,
  'revisionDate': 1654844913000,
  'summonerLevel': 240},
 {'id': '0WFY1F2UK765AELVL5dSy_Erv3Y4n3AIrsipbjbAg_Liaq8',
  'accountId': 'qkQGdFV9h0p9mOAL-tgqU7zwU40R654I6Mf9JMMUqEmeHg',
  'puuid': 'j8rc06BbnWhFDxvPt77h_z2qnymyAzQoNwf_vrjvaUeaRSu3hHVBugEmnIqcuDjaAwzfXICScXIgRQ',
  'name': 'DumbAmerican',
  'profileIconId': 3790,
  'revisionDate': 1548492239000,
  'summonerLevel': 47},
 {'id': 'eXElAearvtCGou9vNFT5K-b

In [9]:
accountIds = [i['accountId'] for i in friends]
puuids = [i['puuid'] for i in friends]

In [6]:
accountIds

['W-SQGFfhtm8nKDogjf7XkKpSAcNwEAbgdWJ2K2ZXAxlTPuk',
 '6itspRrdvLM2APWeg_YbdAkuVd1-3OnNLDPuj1W6JRJ_h1k',
 'qkQGdFV9h0p9mOAL-tgqU7zwU40R654I6Mf9JMMUqEmeHg',
 'YVvUUt-VXK0Ndpz-s4gBEBi5wdqUNDLdRWc31evy6id510s',
 'Z0QTtsE5528PrO7J__TkRXrfstblzxWWfdnvZyu_AQJOI60',
 'ylg-MZsbpCz7oHfqarQ1LR48ZVKmuMqbrjbX5TvpHZNxNJ0',
 'C4UsX_pLTYPUK7LwzMToF6I612mw_3_JZZ8XFDEIe3_mR9s',
 'XEpXFku7dCAoFGDGrifkAqb84zZLdEzRdqveZjucf5QPMtNRRO-XFA6B',
 'GbJ9WSwvaa9991hoR_StFaX0UqD1mdau9FKlIdSYDbHhN9U',
 'njySec2cZ6xEhGGFJ4JfNHwtFNr3zp5BYJD_fiRgYUcMQ2I']

In [10]:
me = puuids[0]

In [11]:
me

'-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw'

In [20]:
lol_watcher.match.matchlist_by_puuid(my_region,me, count=100, queue='ranked_solo_5x5')

HTTPError: 400 Client Error: Bad Request for url: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw/ids?count=100&queue=ranked_solo_5x5

In [18]:
lol_watcher.match.by_id(my_region, 'NA1_4336641349')

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4336641349',
  'participants': ['P3LGitTSWP7JLMjuIoT8BOaLro946okq8R7hpnw3vYPvpwSkcOFJzszCgpcOrii_l0Dn2TN_2aOjKw',
   '2ppRby51sjTmzMoFlYtbqYceAlaQY4zSSM9JfDIhsYpTatQipC944sTTs2M7RJXbJQZKhZ2f9HgU7Q',
   'FcxEMj_mPwBn-FGCCGT0bID0tzulV70_oxirEj5nhR33uQ9fDp806YqTZK7BCuuOK_vIP4oP-gxG9w',
   '-HS6L5RTar48d8s_VPwwgU6vMlxE08uFjHzg8QHAUvspZ6nXZIGgTwY8EGrBb9glYL_buGJXkDlncw',
   'P4B6qgqm64A0x6B0S1s8u8MKgjATETf4OGPaQO0NsZQf21gOzLNHHhxTNoOVbDxQK-Y3JAvcS7UiEw',
   'OR3FqmUObfQVC1o8jv3R8Z7rPS0VFmnGNUZ_ag13CVSA6TIYs6btZsEWwIcVLBGlX0ZA27X21RsyAw',
   '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
   '1bbLN2pK-8qUIq0b4B_k2CDAYRkmRNj2xYK34RSRsO3uXdxZhmv5uS2jl7JfDBWfTQgEOxJdkijPzw',
   'zfH7Cb1QltVjbemgzjS9cD_f3yc2HHFTN6cmHMO-q57KCgeUCyQW0cnZuSWtxwiyBcCo9TNJR0Mu4A',
   'hGFiqH_c8ykOtfmYyn6Hg2WflOVI-J2Nt4HFkhdqB7ClHoCYaX4CFq146wont9KB2E3qiDHIYUv7-Q']},
 'info': {'gameCreation': 1654829072000,
  'gameDuration': 1226,
  